# Neural Networks Sprint Challenge

## 1) Define the following terms:

- Neuron
- Input Layer
- Hidden Layer
- Output Layer
- Activation
- Backpropagation

- Neuron: Receives one or more inputs, weights their values, sums them, and usually applies an activation function. The neural network is made up of layers of neurons, with one layer being an input, another being the output, and all others being hidden layers.
- Input Layer: Layer of neurons which correspond to a row of the independent variables (x) in the data set. The input layer has no inputs going into it in a standard neural network.
- Hidden Layer: Layer of neurons which has inputs going into it and outputs coming out of it. The values of the neurons in these layers are generally difficult to describe functions for.
- Output Layer. Layer of neurons that correspond to a row of dependent variables (y) in the data set. The output layer has no outputs coming from it.
- Activation: a function that is applied to the sum of weighted values coming into a neuron and defines the value of the neuron. Common activations include 'relu' and 'sigmoid' functions.
- Backpropagation: Method of determining the correct weights of the neural network during the fitting of the neural network model. It works throught a generalization of the chain rule for differentials, and uses the calculation of the differential error layer by layer in reverse, from output to input.

## 2) Create a perceptron class that can model the behavior of an AND gate. You can use the following table as your training data:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 1  | 1  | 1  | 1 |
| 1  | 0  | 1  | 0 |
| 0  | 1  | 1  | 0 |
| 0  | 0  | 1  | 0 |

In [3]:
import numpy as np

class NeuralNetwork(object):
    """Simple peceptron class."""
    
    def __init__(self):
        self.syn0 = 2 * np.random.random((3,1)) - 1
        
    def _calc_losses(self, X, y):
        """Run of one epoch in fitting of neural network for training."""
        self.l1 = 1 / (1 + np.exp(-(np.dot(X,self.syn0))))
        self.l1_delta = (y - self.l1) * (1 - self.l1) * self.l1
        self.syn0 += np.dot(X.T, self.l1_delta)
        
    def run(self, X, y, runs=1000):
        """Fits training data for neural network."""
        for i in range(runs):
            self._calc_losses(X, y)
    
    def predict(self, X):
        """Calculate predicted values based on input using fitted model."""
        return 1 / (1 + np.exp(-(np.dot(X,self.syn0))))
            
X = np.array([[1,1,1],[1,0,1],[0,1,1],[0,0,1]])
y = np.array([[1],[0],[0],[0]])

model = NeuralNetwork()
model.run(X,y)
print(model.predict(X))

[[9.33895452e-01]
 [5.55409572e-02]
 [5.55409561e-02]
 [2.44730015e-04]]


## 3) Implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights. 
- Your network must have one hidden layer. 
- You do not have to update weights via gradient descent. You can use something like the derivative of the sigmoid function to update weights.
- Train your model on the Heart Disease dataset from UCI:

[Github Dataset](https://github.com/ryanleeallred/datasets/blob/master/heart.csv)

[Raw File on Github](https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv)


In [110]:
from sklearn.metrics import accuracy_score
import pandas as pd

class MLPNeuralNetwork(object):
    """Multilayer neural network model with configurable layer sizes."""
    def __init__(self, layer_sizes):
        self.layers = []
        for i in range(len(layer_sizes)-1):
            self.layers.append(2 * np.random.random((layer_sizes[i],layer_sizes[i+1])) - 1)
        
    def _calc_losses(self, X, y):
        """Run of one epoch in fitting of neural network for training."""
        self.losses = []
        self.losses.append(1 / (1 + np.exp(-(np.dot(X, self.layers[0])))))
        for layer in self.layers[1:]:
            self.losses.append(1 / (1 + np.exp(-(np.dot(self.losses[-1], layer)))))
        self.loss_delta = [(y - self.losses[-1]) * (1 - self.losses[-1]) * self.losses[-1]]
        for i in range(-2, -len(self.losses) - 1, -1):
            self.loss_delta = [self.loss_delta[0].dot(self.layers[i+1].T) * (1 - self.losses[i]) * self.losses[i]] + self.loss_delta
        self.layers[0] += np.dot(X.T, self.loss_delta[0])
        for i in range(1, len(self.layers)):
            self.layers[i] += np.dot(self.losses[i-1].T, self.loss_delta[i])
        
    def run(self, X, y, runs=1000):
        """Fits training data for neural network."""
        for i in range(runs):
            self._calc_losses(X, y)
    
    def predict(self, X):
        """Calculate predicted values based on input using fitted model."""
        self.losses = []
        self.losses.append(1 / (1 + np.exp(-(np.dot(X, self.layers[0])))))
        for layer in self.layers[1:]:
            self.losses.append(1 / (1 + np.exp(-(np.dot(self.losses[-1], layer)))))
        return self.losses[-1]


df = pd.read_csv("https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv")
X = df.drop(columns=["target"]).values
X = (X - X.mean(axis=0)) / X.std(axis=0)
y = df[["target"]].values

model = MLPNeuralNetwork([X.shape[1], 8, y.shape[1]])
model.run(X, y, 1000)
pred = y.copy()
p = model.predict(X)
pred[p>0.5] = 1
pred[p<=0.5] = 0
print(accuracy_score(y, pred))

0.9801980198019802


## 4) Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy. 

- Use the Heart Disease Dataset (binary classification)
- Use an appropriate loss function for a binary classification task
- Use an appropriate activation function on the final layer of your network. 
- Train your model using verbose output for ease of grading.
- Use GridSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
- When hyperparameter tuning, show you work by adding code cells for each new experiment. 
- Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
- You must hyperparameter tune at least 5 parameters in order to get a 3 on this section.

In [105]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import GridSearchCV

def create_model(layers=[16]):
    model = Sequential()
    for layer in layers:
        model.add(Dense(layer, activation='sigmoid', input_dim=13))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

seed = 7
np.random.seed(seed)

batch_size = [10, 40, 100]
epochs = [10, 50, 100]
layers = [[8], [12], [8,4], [12,4]]
param_grid = dict(batch_size=batch_size, epochs=epochs, layers=layers)
model = KerasClassifier(build_fn=create_model, verbose=1)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, verbose=1)
grid_result = grid.fit(X, y)

print(grid_result.best_score_, grid_result.best_params_)

C:\Users\brit2\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   51.9s
C:\Users\brit2\Anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:  2.4min finished


Epoch 1/10
303/303 [==============================] - 1s 4ms/step - loss: 0.6912 - acc: 0.5578
Epoch 2/10
303/303 [==============================] - 0s 23us/step - loss: 0.6872 - acc: 0.5611
Epoch 3/10
303/303 [==============================] - 0s 30us/step - loss: 0.6837 - acc: 0.5710
Epoch 4/10
303/303 [==============================] - 0s 26us/step - loss: 0.6804 - acc: 0.5809
Epoch 5/10
303/303 [==============================] - 0s 30us/step - loss: 0.6770 - acc: 0.5941
Epoch 6/10
303/303 [==============================] - 0s 23us/step - loss: 0.6740 - acc: 0.5908
Epoch 7/10
303/303 [==============================] - 0s 30us/step - loss: 0.6708 - acc: 0.5908
Epoch 8/10
303/303 [==============================] - 0s 30us/step - loss: 0.6678 - acc: 0.6040
Epoch 9/10
303/303 [==============================] - 0s 26us/step - loss: 0.6651 - acc: 0.6073
Epoch 10/10
303/303 [==============================] - 0s 26us/step - loss: 0.6626 - acc: 0.6139
0.7986798632656387 {'batch_size': 100, '

In [106]:
for i in range(len(grid.cv_results_["mean_test_score"])):
    print(grid.cv_results_["params"][i], grid.cv_results_["mean_test_score"][i])

{'batch_size': 10, 'epochs': 10, 'layers': [8]} 0.5709570984635809
{'batch_size': 10, 'epochs': 10, 'layers': [12]} 0.4653465395221616
{'batch_size': 10, 'epochs': 10, 'layers': [8, 4]} 0.15181518151815182
{'batch_size': 10, 'epochs': 10, 'layers': [12, 4]} 0.6534653473706922
{'batch_size': 10, 'epochs': 50, 'layers': [8]} 0.5676567654798527
{'batch_size': 10, 'epochs': 50, 'layers': [12]} 0.6402640255174228
{'batch_size': 10, 'epochs': 50, 'layers': [8, 4]} 0.557755775921809
{'batch_size': 10, 'epochs': 50, 'layers': [12, 4]} 0.6138613863353288
{'batch_size': 10, 'epochs': 100, 'layers': [8]} 0.6633663402728909
{'batch_size': 10, 'epochs': 100, 'layers': [12]} 0.640264026255104
{'batch_size': 10, 'epochs': 100, 'layers': [8, 4]} 0.6897689799467722
{'batch_size': 10, 'epochs': 100, 'layers': [12, 4]} 0.6963696401111363
{'batch_size': 40, 'epochs': 10, 'layers': [8]} 0.6666666716337204
{'batch_size': 40, 'epochs': 10, 'layers': [12]} 0.3894389534350669
{'batch_size': 40, 'epochs': 10, '